In [1]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image

import torch

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
Setup complete ✅ (36 CPUs, 376.6 GB RAM, 0.1/188.3 GB disk)


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
model = YOLO('yolov8n.pt')

In [4]:
# TRAIN the model on our dataset (data.yml config file) for 10 epochs
results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=10,
   batch=16,
   name='yolov8n_custom')

New https://pypi.org/project/ultralytics/8.0.201 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=Fals

 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 225 layers, 3011043 parameters, 3011027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
WARNING ⚠️ NMS time limit 0.550s exceeded
AMP: checks passed ✅
train: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1/train/labels.cache... 6604 images, 2540 backgrounds, 0 corrupt: 100%|██████████| 9144/9144 [00:00<?, ?it/s]
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1/train/images/Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_0.0_448_448.png: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1/train/images/Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_2.0_448_448.png: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/

: 

In [57]:
# EVALUATE the model's performance on the test set
results = model.val(split='test', save_json=True)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1/test/labels... 1937 images, 818 backgrounds, 0 corrupt: 100%|██████████| 2755/2755 [00:01<00:00, 1590.51it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 345/345 [00:18<00:00, 19.06it/s]
                   all       2755      28167      0.588      0.522      0.466      0.175
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Saving runs/detect/yolov8n_custom122/predictions.json...
Results saved to runs/detect/yolov8n_custom122


In [8]:
# Perform MODEL PREDICTION ont he test set
model.predict(
    model = 'runs/detect/yolov8n_custom93',
    source ='/gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images',
    conf = 0.25
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2755 /gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images/13_0_patch_0.0_0.0_640_640.png: 640x640 6 birds, 5.6ms
image 2/2755 /gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images/13_0_patch_0.0_560.0_640_640.png: 640x640 2 birds, 5.4ms
image 3/2755 /gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images/13_0_patch_560.0_0.0_640_640.png: 640x640 12 birds, 5.2ms
image 4/2755 /gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images/13

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bird'}
 orig_img: array([[[ 40,  40,  40],
         [ 40,  40,  40],
         [ 42,  42,  42],
         ...,
         [ 75,  75,  75],
         [ 72,  72,  72],
         [ 53,  53,  53]],
 
        [[ 43,  43,  43],
         [ 43,  43,  43],
         [ 46,  46,  46],
         ...,
         [ 61,  61,  61],
         [ 62,  62,  62],
         [ 57,  57,  57]],
 
        [[ 44,  44,  44],
         [ 45,  45,  45],
         [ 46,  46,  46],
         ...,
         [ 54,  54,  54],
         [ 61,  61,  61],
         [ 59,  59,  59]],
 
        ...,
 
        [[ 55,  55,  55],
         [ 57,  57,  57],
         [ 52,  52,  52],
         ...,
         [ 56,  56,  56],
         [ 62,  62,  62],
         [103, 103, 103]],
 
        [[ 50,  50,  50],
         [ 52,  52,  52],
         [ 47,  47,  47],
         ...,
         [ 57,  57,  57],
     

In [53]:
# Save PREDICTIONS PREVIEWS

import visualization_utils as visutils
import os

#results = model(source = '/gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images/')
threshold = 0.5
category_id_to_name = {0: 'bird'}
detection_boxes = []

for result in results:
    im_array = result.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    #im.show()  # show image
    im.save('/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/yolov8n_custom12/predictions/' + os.path.basename(result.path).split('.png')[0] + '.jpg')  # save image
